*   Nama : **Virgiawan Teguh Kusuma**
*   Email : **181240000833@unisnu.ac.id**
*   Domisili : **Kabupaten Jepara, Jawa Tengah**

In [1]:
# import pandas dan membuat dataframe
import pandas as pd
dataframe = pd.read_csv('Womens Clothing E-Commerce Reviews.csv')

FileNotFoundError: ignored

In [ ]:
# drop kolom yang tidak dibutuhkan
dataframe = dataframe.drop(columns=
                           ['Unnamed: 0',
                            'Clothing ID', 
                            'Age', 
                            'Title', 
                            'Rating', 
                            'Recommended IND', 
                            'Positive Feedback Count',
                            'Division Name',
                            'Department Name'
                            ]
                           )

In [ ]:
dataframe

,Review Text,Class Name
0,Absolutely wonderful - silky and sexy and comf...,Intimates
1,Love this dress! it's sooo pretty. i happene...,Dresses
2,I had such high hopes for this dress and reall...,Dresses
3,"I love, love, love this jumpsuit. it's fun, fl...",Pants
4,This shirt is very flattering to all due to th...,Blouses
...,...,...
23481,I was very happy to snag this dress at such a ...,Dresses
23482,"It reminds me of maternity clothes. soft, stre...",Knits
23483,"This fit well, but the top was very see throug...",Dresses
23484,I bought this dress for a wedding i have this ...,Dresses


In [ ]:
# rename agar tidak ada spasi
dataframe = dataframe.rename(columns={'Class Name' : 'Classname'})

# one hot encoding
category = pd.get_dummies(dataframe.Classname)
newdataframe = pd.concat([dataframe, category], axis=1)
newdataframe = newdataframe.drop(columns=['Classname'])

In [ ]:
newdataframe.head()

,Review Text,Blouses,Casual bottoms,Chemises,Dresses,Fine gauge,Intimates,Jackets,Jeans,Knits,Layering,Legwear,Lounge,Outerwear,Pants,Shorts,Skirts,Sleep,Sweaters,Swim,Trend
0,Absolutely wonderful - silky and sexy and comf...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Love this dress! it's sooo pretty. i happene...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,I had such high hopes for this dress and reall...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"I love, love, love this jumpsuit. it's fun, fl...",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,This shirt is very flattering to all due to th...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# mengubah nilai dari dataframe ke numpy array
reviewtext = newdataframe['Review Text'].astype(str).values
label = newdataframe[newdataframe.columns[1:21]].values
# label = newdataframe[newdataframe.columns[1:10]].values

In [ ]:
# split data training dan testing
from sklearn.model_selection import train_test_split
reviewtext_train, reviewtext_test, label_train, label_test = train_test_split(reviewtext, label, test_size=0.2)

In [ ]:
# import tokenizer & pad sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
# mengubah kata ke bilangan numerik
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(reviewtext_train) 
tokenizer.fit_on_texts(reviewtext_test)
 
# convert menjadi sequence
sequence_train = tokenizer.texts_to_sequences(reviewtext_train)
sequence_test = tokenizer.texts_to_sequences(reviewtext_test)
padded_train = pad_sequences(sequence_train) 
padded_test = pad_sequences(sequence_test)

In [ ]:
# import tensorflow & dropout
import tensorflow as tf
from tensorflow.keras.layers import Dropout

# embedding, lstm, model sequential
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(20, activation='softmax'),
])

# menggunakan loss categorical_crossentropy & optimizer adam
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# menggunakan callback agar pengujian berhenti jika akurasi mencapai lebih dari 90%
class callback(tf.keras.callbacks.Callback) :
  def on_epoch_end(self, epoch, logs={}) :
    if(logs.get('accuracy') > 0.9) :
      self.model.stop_training = True

In [ ]:
# train model dengan fit
history = model.fit(padded_train, label_train, epochs=100, 
                    validation_data=(padded_test, label_test), batch_size=128, verbose=2, callbacks=callback())

Epoch 1/100
147/147 - 20s - loss: 0.3559 - accuracy: 0.8945 - val_loss: 2.8381 - val_accuracy: 0.5502
Epoch 2/100
147/147 - 21s - loss: 0.3224 - accuracy: 0.9040 - val_loss: 2.9746 - val_accuracy: 0.5562
